In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder
from joblib import dump

In [11]:
def preprocess_data(df, label_encoders=None, is_training=True):
    df['travel_date'] = pd.to_datetime(df['travel_date'], format='%d-%m-%y')
    df['Year'] = df['travel_date'].dt.year
    df['Month'] = df['travel_date'].dt.month
    df['Day'] = df['travel_date'].dt.day
    df['Hour'] = pd.to_datetime(df['travel_time'], format='%H:%M').dt.hour
    df['Minutes'] = pd.to_datetime(df['travel_time'], format='%H:%M').dt.minute
    df['number_of_ticket'] = df.groupby(
        ['travel_from', 'car_type', 'max_capacity', 'Year', 'Month', 'Day', 'Hour', 'Minutes']
    )['travel_from'].transform('count')

    if is_training:

        label_encoders = {
            'travel_from': LabelEncoder(),
            'car_type': LabelEncoder()
        }
        df['travel_from'] = label_encoders['travel_from'].fit_transform(df['travel_from'])
        df['car_type'] = label_encoders['car_type'].fit_transform(df['car_type'])


        dump(label_encoders['travel_from'], 'travel_from_encoder.joblib')
        dump(label_encoders['car_type'], 'car_type_encoder.joblib')
    else:

        df['travel_from'] = label_encoders['travel_from'].transform(df['travel_from'])
        df['car_type'] = label_encoders['car_type'].transform(df['car_type'])

    return df

In [12]:

def standardize_data(X, is_training=True, mean=None, std_dev=None):
    if is_training:

        mean = np.mean(X, axis=0)
        std_dev = np.std(X, axis=0)
        np.save('mean.npy', mean)
        np.save('std_dev.npy', std_dev)

    X = (X - mean) / std_dev
    return X, mean, std_dev

In [13]:
def train_model(X, Y):
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    model = RandomForestRegressor()
    model.fit(x_train, y_train)

    importances = model.feature_importances_
    importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importances}).sort_values(by='Importance', ascending=False)

    selector = SelectFromModel(model, threshold=0.03, prefit=True)
    X_selected = selector.transform(X)

    y_pred = model.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)
    model_score = model.score(x_train, y_train)

    return model, importance_df, X_selected, mse, model_score


In [14]:
if __name__ == "__main__":
    df = pd.read_csv('train_revised.csv')


    df = preprocess_data(df)
    X = df[['travel_from', 'car_type', 'max_capacity', 'Year', 'Month', 'Day', 'Hour', 'Minutes']]
    Y = df['number_of_ticket']


    X, mean, std_dev = standardize_data(X, is_training=True)


    model, importance_df, X_selected, mse, model_score = train_model(X, Y)


    dump(model, 'random_forest_model.joblib')

    print(f"Mean Squared Error: {mse}")
    print(f"Model Accuracy: {model_score}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Mean Squared Error: 0.8569708490657373
Model Accuracy: 0.9990385439332746
